# Artificial and Computational Intelligence Assignment 2

## 3-Player TIC-TAC-TOE using MIN-MAX ALGRORITHM with ALPHA BETA PRUNING

**Code starts here**

In [1]:
import math
import random

In [2]:
# Constants
ROWS = 6
COLS = 6
EMPTY = '.'
DEPTH = 4  # Depth for the Minimax algorithm

In [3]:
# Initialize the board
def create_board():
    return [[EMPTY for _ in range(COLS)] for _ in range(ROWS)]

In [4]:
# Display the board
def display_board(board):
    i = 0
    for row in board:
        print(i, end=' ')
        i += 1
        print(' '.join(row))
    print(' ', end=' ')
    print(' '.join([str(i) for i in range(COLS)]))

In [5]:
# Check if a position is valid
def is_valid_position(board, row, col):
    return board[row][col] == EMPTY

In [6]:
# Place a piece on the board
def drop_piece(board, row, col, piece):
    board[row][col] = piece

In [7]:
# Check for a win
def is_winning_move(board, piece):
    # Check horizontal
    for row in range(ROWS):
        for col in range(COLS - 3):
            if all(board[row][col + i] == piece for i in range(4)):
                return True

    # Check vertical
    for row in range(ROWS - 3):
        for col in range(COLS):
            if all(board[row + i][col] == piece for i in range(4)):
                return True

    # Check positively sloped diagonal
    for row in range(ROWS - 3):
        for col in range(COLS - 3):
            if all(board[row + i][col + i] == piece for i in range(4)):
                return True

    # Check negatively sloped diagonal
    for row in range(3, ROWS):
        for col in range(COLS - 3):
            if all(board[row - i][col + i] == piece for i in range(4)):
                return True

    return False

In [8]:
# Check if the board is full
def is_full(board):
    return all(board[row][col] != EMPTY for row in range(ROWS) for col in range(COLS))

In [9]:
# Evaluate a window (4 cells) for scoring
def evaluate_window(window, piece):
    score = 0
    opp_pieces = [p for p in ['X', 'O', 'T'] if p != piece]

    # Scoring for the AI's pieces
    if window.count(piece) == 4:
        score += 1000  # Winning
    if window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 100  # Prioritize a potential win
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 10  # Building towards a win

    # Scoring for opponents' pieces (to block them)
    for opp_piece in opp_pieces:
        if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
            score -= 200  # Strong penalty to block opponents' win
        elif window.count(opp_piece) == 2 and window.count(EMPTY) == 2:
            score -= 20  # Moderate penalty to slow down opponent

    return score

In [10]:
# Score the board for a given player
def score_position(board, piece):
    score = 0

    # Horizontal scoring
    for row in range(ROWS):
        row_array = board[row]
        for col in range(COLS - 3):
            window = row_array[col:col + 4]
            score += evaluate_window(window, piece)

    # Vertical scoring
    for col in range(COLS):
        col_array = [board[row][col] for row in range(ROWS)]
        for row in range(ROWS - 3):
            window = col_array[row:row + 4]
            score += evaluate_window(window, piece)

    # Positive diagonal scoring
    for row in range(ROWS - 3):
        for col in range(COLS - 3):
            window = [board[row + i][col + i] for i in range(4)]
            score += evaluate_window(window, piece)

    # Negative diagonal scoring
    for row in range(3, ROWS):
        for col in range(COLS - 3):
            window = [board[row - i][col + i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score

In [11]:
# Get all valid positions
def get_valid_positions(board):
    return [(row, col) for row in range(ROWS) for col in range(COLS) if board[row][col] == EMPTY]

In [12]:
# Minimax with alpha-beta pruning
def minimax(board, depth, alpha, beta, maximizing_player, piece):
    valid_positions = get_valid_positions(board)
    is_terminal = is_full(board) or any(is_winning_move(board, p) for p in ['X', 'O', 'T'])

    if depth == 0 or is_terminal:
        if is_terminal:
            if is_winning_move(board, piece):
                return (None, 1000000)
            elif any(is_winning_move(board, p) for p in ['X', 'O', 'T'] if p != piece):
                return (None, -1000000)
            else:
                return (None, 0)
        else:
            return (None, score_position(board, piece))

    if maximizing_player:
        value = -math.inf
        best_position = random.choice(valid_positions)
        for position in valid_positions:
            row, col = position
            temp_board = [row[:] for row in board]
            drop_piece(temp_board, row, col, piece)
            _, new_score = minimax(temp_board, depth - 1, alpha, beta, False, piece)
            if new_score > value:
                value = new_score
                best_position = position
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return best_position, value
    else:
        value = math.inf
        best_position = random.choice(valid_positions)
        opp_pieces = [i for i in ['X', 'O', 'T'] if i != piece]
        for opp_piece in opp_pieces:
            for position in valid_positions:
                row, col = position
                temp_board = [row[:] for row in board]
                drop_piece(temp_board, row, col, opp_piece)
                _, new_score = minimax(temp_board, depth - 1, alpha, beta, True, piece)
                if new_score < value:
                    value = new_score
                    best_position = position
                beta = min(beta, value)
                if alpha >= beta:
                    break
        return best_position, value

In [13]:
# Main game loop
def play_game():
    board = create_board()
    game_over = False
    turn = 0
    players = ['X', 'O', 'T']

    print("Welcome to 3-Player Tic-Tac-Toe!")
    print("This game has 3 players: X, O, T. X goes first, O goes second, T goes third.")
    chosen_piece = input("Enter the piece which you would like to play as (X, O or T): ").strip()[0].upper()
    while chosen_piece not in players:
        chosen_piece = input("Invalid input! Please choose a valid player (X, O or T): ").strip()[0].upper()
    print(f"You will be playing as '{chosen_piece}'.")
    display_board(board)

    while not game_over:
        current_player = players[turn % 3]
        if current_player == chosen_piece:
            print("Valid Positions :", get_valid_positions(board))
            row, col = map(int, input("Your turn (row col): ").split())
            while not is_valid_position(board, row, col):
                row, col = map(int, input("Invalid position. Choose again (row col): ").split())
        else:
            print(f"Computer {current_player}'s turn.")
            (row, col), _ = minimax(board, DEPTH, -math.inf, math.inf, True, current_player)

        drop_piece(board, row, col, current_player)
        display_board(board)

        if is_winning_move(board, current_player):
            if current_player == chosen_piece:
                print("Congratulations! You win!")
            else:
                print(f"Computer {current_player} wins!")
            game_over = True

        turn += 1
        if is_full(board) and not game_over:
            print("It's a draw!")
            game_over = True

In [15]:
# Run the game
play_game()

Welcome to 3-Player Tic-Tac-Toe!
This game has 3 players: X, O, T. X goes first, O goes second, T goes third.
Enter the piece which you would like to play as (X, O or T): X
You will be playing as 'X'.
0 . . . . . .
1 . . . . . .
2 . . . . . .
3 . . . . . .
4 . . . . . .
5 . . . . . .
  0 1 2 3 4 5
Valid Positions : [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
Your turn (row col): 3 0
0 . . . . . .
1 . . . . . .
2 . . . . . .
3 X . . . . .
4 . . . . . .
5 . . . . . .
  0 1 2 3 4 5
Computer O's turn.
0 O . . . . .
1 . . . . . .
2 . . . . . .
3 X . . . . .
4 . . . . . .
5 . . . . . .
  0 1 2 3 4 5
Computer T's turn.
0 O . . . . .
1 . . . . . .
2 . T . . . .
3 X . . . . .
4 . . . . . .
5 . . . . . .
  0 1 2 3 4 5
Valid Positions : [(0, 1), (0, 2), (0